In [166]:
from pyspark.sql import SparkSession

spark = SparkSession. \
    builder. \
    config("spark.ui.port", "0"). \
    enableHiveSupport(). \
    appName("Spark broadcast join"). \
    master("yarn"). \
    config('spark.executor.instances','3'). \
    config('spark.executor.memory','3gb'). \
    config('spark.executor.cores','4'). \
    config('spark.dynamicAllocation.enabled','False'). \
    getOrCreate()

In [165]:
spark.stop()

In [167]:
spark.sparkContext.applicationId

'application_1745651200635_9211'

In [145]:
 #Disable AQE and Broadcast join

spark.conf.set("spark.sql.adaptive.enabled", False)
spark.conf.set("spark.sql.adaptive.coalescePartitions.enabled", False)
spark.conf.set("spark.sql.autoBroadcastJoinThreshold", -1)


In [146]:
# Read Employee data
schema = "first_name string, last_name string, job_title string, dob string, email string, phone string, salary double, department_id int"

emp = spark.read.format("csv").schema(schema).option("header", True).load("Datasets/employee_records.csv")

In [147]:
# Read DEPT CSV data
dept_schema = "department_id int, department_name string, description string, city string, state string, country string"

dept = spark.read.format("csv").schema(dept_schema).option("header", True).load("Datasets/department_data.csv")


In [148]:
# Join Datasets

df_joined = emp.join(dept, on=emp.department_id==dept.department_id, how="left_outer")


In [149]:
#write Action

df_joined.write.format("noop").mode("overwrite").save()

In [150]:
#check the Execution Plan

df_joined.explain()

== Physical Plan ==
SortMergeJoin [department_id#1653], [department_id#1662], LeftOuter
:- *(1) Sort [department_id#1653 ASC NULLS FIRST], false, 0
:  +- Exchange hashpartitioning(department_id#1653, 200), ENSURE_REQUIREMENTS, [id=#1699]
:     +- FileScan csv [first_name#1646,last_name#1647,job_title#1648,dob#1649,email#1650,phone#1651,salary#1652,department_id#1653] Batched: false, DataFilters: [], Format: CSV, Location: InMemoryFileIndex[hdfs://m01.itversity.com:9000/user/itv018960/Datasets/employee_records.csv], PartitionFilters: [], PushedFilters: [], ReadSchema: struct<first_name:string,last_name:string,job_title:string,dob:string,email:string,phone:string,s...
+- *(3) Sort [department_id#1662 ASC NULLS FIRST], false, 0
   +- Exchange hashpartitioning(department_id#1662, 200), ENSURE_REQUIREMENTS, [id=#1710]
      +- *(2) Filter isnotnull(department_id#1662)
         +- FileScan csv [department_id#1662,department_name#1663,description#1664,city#1665,state#1666,country#1667] Batche

In [198]:
#enable broadcastJoin with the help of AQE

spark.conf.set("spark.sql.adaptive.enabled", True)
spark.conf.set("spark.sql.adaptive.coalescePartitions.enabled", True)
spark.conf.set("spark.sql.autoBroadcastJoinThreshold", "10MB")

In [169]:
#Join again

df_bj_joined = emp.join(dept, on=emp.department_id==dept.department_id, how="left_outer")

In [ ]:
#write Action

df_bj_joined.write.format("noop").mode("overwrite").save()

In [199]:
#check the explain plan

df_bj_joined.explain()

== Physical Plan ==
*(2) BroadcastHashJoin [department_id#1653], [department_id#1662], LeftOuter, BuildRight, false
:- FileScan csv [first_name#1646,last_name#1647,job_title#1648,dob#1649,email#1650,phone#1651,salary#1652,department_id#1653] Batched: false, DataFilters: [], Format: CSV, Location: InMemoryFileIndex[hdfs://m01.itversity.com:9000/user/itv018960/Datasets/employee_records.csv], PartitionFilters: [], PushedFilters: [], ReadSchema: struct<first_name:string,last_name:string,job_title:string,dob:string,email:string,phone:string,s...
+- BroadcastExchange HashedRelationBroadcastMode(List(cast(input[0, int, false] as bigint)),false), [id=#2376]
   +- *(1) Filter isnotnull(department_id#1662)
      +- FileScan csv [department_id#1662,department_name#1663,description#1664,city#1665,state#1666,country#1667] Batched: false, DataFilters: [isnotnull(department_id#1662)], Format: CSV, Location: InMemoryFileIndex[hdfs://m01.itversity.com:9000/user/itv018960/Datasets/department_data.csv], 

In [200]:
#enable broadcastJoin with the help of AQE

spark.conf.set("spark.sql.autoBroadcastJoinThreshold", "100MB")
spark.conf.get("spark.sql.autoBroadcastJoinThreshold")

'100MB'

In [201]:
#self Join
schema1 = "first_name string, last_name string, job_title string, dob string, email string, phone string, salary double, department_id int"

emp1 = spark.read.format("csv").schema(schema1).option("header", True).load("Datasets/employee_records.csv")

emp_df = emp1.union(emp1).union(emp1)

df_self_bj_join = emp_df.join(emp, on=emp.department_id==emp_df.department_id, how="left_outer")

In [202]:
#check the explain plan

df_self_bj_join.explain()

== Physical Plan ==
AdaptiveSparkPlan isFinalPlan=false
+- BroadcastHashJoin [department_id#2313], [department_id#1653], LeftOuter, BuildRight, false
   :- Union
   :  :- FileScan csv [first_name#2306,last_name#2307,job_title#2308,dob#2309,email#2310,phone#2311,salary#2312,department_id#2313] Batched: false, DataFilters: [], Format: CSV, Location: InMemoryFileIndex[hdfs://m01.itversity.com:9000/user/itv018960/Datasets/employee_records.csv], PartitionFilters: [], PushedFilters: [], ReadSchema: struct<first_name:string,last_name:string,job_title:string,dob:string,email:string,phone:string,s...
   :  :- FileScan csv [first_name#2306,last_name#2307,job_title#2308,dob#2309,email#2310,phone#2311,salary#2312,department_id#2313] Batched: false, DataFilters: [], Format: CSV, Location: InMemoryFileIndex[hdfs://m01.itversity.com:9000/user/itv018960/Datasets/employee_records.csv], PartitionFilters: [], PushedFilters: [], ReadSchema: struct<first_name:string,last_name:string,job_title:string,dob:st

In [ ]:
#write Action

df_self_bj_join.write.format("noop").mode("overwrite").save()

In [195]:
#check the explain plan

df_self_bj_join.explain()

== Physical Plan ==
AdaptiveSparkPlan isFinalPlan=false
+- BroadcastHashJoin [department_id#2233], [department_id#1653], LeftOuter, BuildRight, false
   :- Union
   :  :- FileScan csv [first_name#2226,last_name#2227,job_title#2228,dob#2229,email#2230,phone#2231,salary#2232,department_id#2233] Batched: false, DataFilters: [], Format: CSV, Location: InMemoryFileIndex[hdfs://m01.itversity.com:9000/user/itv018960/Datasets/employee_records.csv], PartitionFilters: [], PushedFilters: [], ReadSchema: struct<first_name:string,last_name:string,job_title:string,dob:string,email:string,phone:string,s...
   :  :- FileScan csv [first_name#2226,last_name#2227,job_title#2228,dob#2229,email#2230,phone#2231,salary#2232,department_id#2233] Batched: false, DataFilters: [], Format: CSV, Location: InMemoryFileIndex[hdfs://m01.itversity.com:9000/user/itv018960/Datasets/employee_records.csv], PartitionFilters: [], PushedFilters: [], ReadSchema: struct<first_name:string,last_name:string,job_title:string,dob:st

In [190]:
#write Action

df_self_bj_join.write.format("noop").mode("overwrite").save()

Py4JJavaError: An error occurred while calling o1766.save.
: org.apache.spark.sql.catalyst.errors.package$TreeNodeException: execute, tree:
Exchange hashpartitioning(department_id#1653, 200), ENSURE_REQUIREMENTS, [id=#2298]
+- FileScan csv [first_name#1646,last_name#1647,job_title#1648,dob#1649,email#1650,phone#1651,salary#1652,department_id#1653] Batched: false, DataFilters: [], Format: CSV, Location: InMemoryFileIndex[hdfs://m01.itversity.com:9000/user/itv018960/Datasets/employee_records.csv], PartitionFilters: [], PushedFilters: [], ReadSchema: struct<first_name:string,last_name:string,job_title:string,dob:string,email:string,phone:string,s...

	at org.apache.spark.sql.catalyst.errors.package$.attachTree(package.scala:56)
	at org.apache.spark.sql.execution.exchange.ShuffleExchangeExec.doExecute(ShuffleExchangeExec.scala:163)
	at org.apache.spark.sql.execution.SparkPlan.$anonfun$execute$1(SparkPlan.scala:180)
	at org.apache.spark.sql.execution.SparkPlan.$anonfun$executeQuery$1(SparkP

In [19]:
#stop the spark session

spark.stop()